In [ ]:
import natsort
import numpy as np
import pandas as pd
import param
import plotly.express as px
import plotly.graph_objects as go
import re
import traceback
import panel as pn
pn.extension("plotly")
from io import BytesIO
from io import StringIO
from bokeh.models.widgets.tables import NumberFormatter
from sklearn.decomposition import PCA

In [2]:
#e.findall('(\D+)',k)
#isteEe = ["02", "33", "3"]
#items.zfill(2) for items in listeEe]
#e.findall('(\d+)',k)[0].zfill(2)
#=[[re.match(name_pattern, col).group("frac")] for col in df_index_LFQ.columns]
#isteeeee = list(list(zip(*k))[0])
#items.zfill(3) for items in listeeeee]
#items.zfill(2) for items in [re.match(name_pattern, col).group("frac")] for col in df_index_LFQ.columns]

In [3]:
#stringg= ["Ratio H/L count MAP1_03K", "Ratio H/L variability [%] MAP1_03K", "Ratio H/L count MAP1_03K", "Ratio H/L variability [%] MAP1_03K"]
#stringg = LFQ intensity Control_Cyt_1
#[re.findall(sets_SILAC, stringg) for col in stringg]
#k=re.match(".* (?P<rep>.*)_(?P<frac>.*)", stringg).group("rep")
#k
#[[s for s in sets_SILAC if col.startswith(s)][0] for col in stringg]
#[re.match(name_pattern_SILAC, col).group("frac")]
#[re.match("(?P<sets>.*) (?P<rep>.*)_(?P<frac>.*)", col).group("rep") for col in stringg]
#[re.match("(?P<sets>.*) (?P<rep>.*)_(?P<frac>.*)", col).group("sets") for col in stringg]


#bool([re.match(name_pattern_SILAC, col) for col in i_class.df_original.columns])
#any([[col.startswith(s) for s in sets_SILAC] for col in i_class.df_original.columns])

#[items.zfill(3) for items in ["03K", "12K"]]

#bool([re.match(name_pattern_SILAC, col) for col in i_class.df_original.columns])

#[re.match(name_pattern_SILAC, col).group("sets") for col in df_index_SILAC.columns]
#[col for col in df_index.columns if [re.match(name_pattern_SILAC, col).group("") for s in sets_SILAC]]
#len([[s for s in sets_LFQ if re.match(name_pattern_SILAC, col)][0] for col in df_index.columns])

In [42]:
class SpatialDataSet:

    def __init__(self, **kwargs):
        """Import of the raw file of interest. Dataframe will be generated, which contains only the data of desired
        column names, specified by the dictionary entry regex["col_shortened"]

        Dictionaries are created, that ate used for filtering and plotting, respectively.

        Args:
            filename: raw file obtained by the LFQ/SILAC approach (Protein Groups Files), processed by MaxQuant

        Returns:
            df_original: shortened data frame, contains only the information, which was determined due to
            the values of the key "col_shortened"
        """
        # df_original contains all information of the raw file; tab separated file is imported,
        # without considering comments, marked with #


        #self.filename = "LFQ_proteinGroups.txt" if "filename" not in kwargs.keys() else kwargs["filename"]
        #self.filename = "proteinGroupsCOVID.txt" if "filename" not in kwargs.keys() else kwargs["filename"]
        #self.filename = "proteinGroups_LFQ_Deep_3_Maps.txt" if "filename" not in kwargs.keys() else kwargs["filename"]
        self.filename = "6_deep_maps.txt" if "filename" not in kwargs.keys() else kwargs["filename"]
        
        
            
        self.map_of_interest = "MAP1" if "map_of_interest" not in kwargs.keys() else kwargs["map_of_interest"]
        self.cluster_of_interest = "Proteasome" if "cluster_of_interest" not in kwargs.keys() else kwargs[
            "cluster_of_interest"]

        self.map_index = ".*[ .](.*)_.*K" if "map_index" not in kwargs.keys() else kwargs["map_index"]
        self.fraction_index = ".*_(\d+[Kk])" if "fraction_index" not in kwargs.keys() else kwargs["fraction_index"]

        self.summed_MSMS_counts = 2 if "summed_MSMS_counts" not in kwargs.keys() else kwargs["summed_MSMS_counts"]
        self.consecutive_LFQ_I = 4 if "consecutive_LFQ_I" not in kwargs.keys() else kwargs["consecutive_LFQ_I"]

        # self.lfq_filter_param = {"summed MS/MS counts": 2, "consecutive LFQ_I": 4}

        self.regex = {
            "imported_columns": "^[Rr]atio H/L (?!normalized|type|is.*).+|id$|[Mm][Ss].*[cC]ount.+$|[Ll][Ff][Qq].*|.*[nN]ames.*|.*[Pp][rR].*[Ii][Dd]s.*|[Pp]otential.[cC]ontaminant|[Oo]nly.[iI]dentified.[bB]y.[sS]ite|[Rr]everse|[Ss]core|[Qq]-[Vv]alue",
            "index_col_silac": "[Pp]rotein.[Ii][Dd]s|[Mm]ajority.[Pp]rotein.[Ii][Dd]s|[Pp]rotein.[Nn]ames|[Gg]ene.[Nn]ames|[Ii][Dd]|[Ss]core|[Qq]-[Vv]alue",
            "index_col_lfq": ".*[Pp][rR].*[Ii][Dd]s.*|.*[nN]ames.*|[Ii][Dd]|[Ss]core|[Qq]-[Vv]alue|MS/MS.count$",

            "type_count_silac": "([Rr]atio.[Hh]/[Ll].[cC]ount)[ .].*_.*K",
            "type_var_silac": "([Rr]atio.[Hh]/[Ll].[Vv]ariability....)[ .].*_.*K",
            "type_ratio_silac": "([rR]atio.[Hh]/[Ll])[ .](?![cC]ount|[Vv]ariability).*_.*K",

            "type_count_lfq": "([Rr]atio.[Hh]/[Ll].[cC]ount)[ .].*_.*K",
            "type_var_lfq": "([Rr]atio.[Hh]/[Ll].[Vv]ariability....)[ .].*_.*K",
            "type_ratio_silac": "([rR]atio.[Hh]/[Ll])[ .](?![cC]ount|[Vv]ariability).*_.*K",

            "type_msms_lfq": "([Mm][Ss]/[Mm][Ss].[cC]ount)[ .].*_.*K",
            "type_intensity_lfq": "([Ll][Ff][Qq].[Ii]ntensity)[ .].*_.*K",

            # "type_lfq": "(.*[nNTt]{1}[yYtT]{1})[ .].*_\d+[Kk]$",

            "lfq_nan": "[Ll][Ff][Qq].*",

            "contaminants": "[Pp]otential.[cC]ontaminant",
            "sites": "[Oo]nly.[iI]dentified.[bB]y.[sS]ite",
            "reverse": "[Rr]everse"
        }

    
        #self.acquisition = "SILAC" if "acquisition" not in kwargs.keys() else kwargs["acquisition"]
        self.acquisition = "LFQ" if "acquisition" not in kwargs.keys() else kwargs["acquisition"]


    def data_reading(self):
        """Data import.

        Args:
            filename: stored as attribute

        Returns:
            df_orginal: raw, unprocessed dataframe, single level column index
        """

        self.df_original = pd.read_csv(self.filename, sep="\t", comment="#",
                                       usecols=lambda x: bool(re.match(self.regex["imported_columns"], x)))

        
        return self.df_original
    
    def indexingdf_lfq(self):
        
        # deep copy of the dataframe
        df_original = self.df_original.copy()
    
        # iteration through the column names, and searches for column names that are mentioned in the dictionary
        # entry regex["index_col_lfq"]
        col_to_index = [col for col in df_original.columns if re.match(regex["index_col_lfq"], col)]
    
        # column names are defined as index
        df_index = df_index.set_index(col_to_index)
    
        # all other columns, that do not start with "LFQ" and "MS" (essentially "Only identified by site",
        # "Reverse", "Potential contaminant") are removed
        df_index = df_index.drop([column for column in df_index.columns
                                    if not column.startswith(("LFQ", "MS"))], axis=1)
    
        # "LFQ intensity"-values: converting 0 into NAN
        # iteration through the column names, and searches for column names (LFQ intensity, deposited as regular
        # expression in the dictionary entry regex["lfq_nan"]
        df_index[[col for col in df_index.columns if re.match(regex["lfq_nan"], col)]] = \
            df_index[[col for col in df_index.columns if re.match(regex["lfq_nan"], col)]].replace(0, np.nan)
    
        # multindex will be generated, by isolating the information about the Map, Fraction and Type from each
        # individual column name
        # names=["Map", "Fraction", "Type"] defines the label of the multiindex
    
        desired_columns_types = [e1 for e1 in [
            re.findall(regex["type_msms_lfq"], column) or re.findall(regex["type_intensity_lfq"], column) for column
            in df_index.columns] if e1]
        col_list_type_lfq = [item for sublist in desired_columns_types for item in sublist]
    
        columns = pd.MultiIndex.from_arrays([
            [re.findall(map_index, column)[0] for column in df_index.columns],
            [re.findall(fraction_index, column)[0] for column in df_index.columns],
            col_list_type_lfq],
            names=["Map", "Fraction", "Type"])
        df_index.columns = columns
    
        return df_index

sets_LFQ = ["LFQ intensity", "MS/MS count", "Intensity"]
#sets_SILAC = ["Ratio H/L variability [%]", "Ratio H/L count", "Ratio H/L"]
sets_SILAC = ["([Rr]atio.[Hh]/[Ll].[Vv]ariability....)", "([Rr]atio.[Hh]/[Ll].[cC]ount)", "([Rr]atio.[Hh]/[Ll])"]
name_pattern_LFQ = ".* (?P<cond>.*)_(?P<rep>.*)_(?P<frac>.*)"
#name_pattern_SILAC = ".* (?P<rep>.*)_(?P<frac>.*)"
name_pattern_SILAC = "(?P<sets>.*) (?P<rep>.*)_(?P<frac>.*)"
fraction_dict = {"1K": "01K","3K": "03K", "6K": "06K", "12K": "12K", "24K": "24K", "80K": "80K", "01K": "01K","03K": "03K", "06K": "06K", "012K": "12K", "024K": "24K", "080K": "80K", "Cyt": "Cyt", "Mem": "Mem", "Nuc": "Nuc", "Prot": "Prot", "cyt": "Cyt"}

In [109]:
test_liast = ['Ratio H/L MAP6_80K', 'Ratio H/L variability [%] MAP1_03K', 'Ratio H/L count MAP1_03K', 'Ratio H/L variability [%] MAP1_06K', 'Ratio H/L count MAP1_06K']
sets_SILAC = ["([Rr]atio.[Hh]/[Ll].[Vv]ariability....)", "([Rr]atio.[Hh]/[Ll].[cC]ount)", "([Rr]atio.[Hh]/[Ll])"]
#sets_SILAC = ["([Rr]atio.[Hh]/[Ll].[Vv]ariability.[%]).*"]
#sets_SILAC = ["Ratio H/L variability [%]", "Ratio H/L count", "Ratio H/L"]
#"type_count_silac": "([Rr]atio.[Hh]/[Ll].[cC]ount)[ .].*_.*K",
#            "type_var_silac": "([Rr]atio.[Hh]/[Ll].[Vv]ariability....)[ .].*_.*K",
#            "type_ratio_silac": "([rR]atio.[Hh]/[Ll])[ .](?![cC]ount|[Vv]ariability).*_.*K",
[[re.findall(s, col)[0] for s in sets_SILAC if re.match(s,col)] for col in test_liast]

#negative lookaheadf
#if len re.findall
#([[re.match(s, col) for s in sets_SILAC] for col in test_liast])

[['Ratio H/L'],
 ['Ratio H/L variability [%]', 'Ratio H/L'],
 ['Ratio H/L count', 'Ratio H/L'],
 ['Ratio H/L variability [%]', 'Ratio H/L'],
 ['Ratio H/L count', 'Ratio H/L']]

In [4]:
i_file = pn.widgets.Select(options=["6_deep_maps.txt","LFQ_proteinGroups.txt","proteinGroupsCOVID.txt", 
                                              "proteinGroups_LFQ_Deep_3_Maps.txt"])
i_file

#i_acquisition = pn.widgets.Select(
#    value=_countries[continent.value][0], 
#    options=_countries[continent.value]
#)
#
#@pn.depends(continent.param.value, watch=True)
#def _update_countries(continent):
#    countries = _countries[continent]
#    country.options = countries
#    country.value = countries[0]

Select(options=['6_deep_maps.txt', ...], value='6_deep_maps.txt')

In [95]:
#@pn.depends(i_file.param.value)
#def read_file(file):

i_class = SpatialDataSet()
i_class.filename= i_file.value
i_class.data_reading()

df_index = i_class.df_original.set_index([col for col in i_class.df_original.columns if any([re.match(s, col) for s in sets_LFQ]) == False])
#df_index = i_class.df_original.set_index([col for col in i_class.df_original.columns if any([re.match(s, col) for s in sets_SILAC]) == False])

df_index
#pn.Row(i_file, read_file)

C:\Users\valbrecht\Anaconda3\envs\QC repository\lib\site-packages\IPython\core\interactiveshell.py:3263: DtypeWarning: Columns (269,270) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


,,,,,,,,,,LFQ intensity EGF_rep1_03K,LFQ intensity EGF_rep1_06K,LFQ intensity EGF_rep1_12K,LFQ intensity EGF_rep1_24K,LFQ intensity EGF_rep1_80K,LFQ intensity EGF_rep2_03K,LFQ intensity EGF_rep2_06K,LFQ intensity EGF_rep2_12K,LFQ intensity EGF_rep2_24K,LFQ intensity EGF_rep2_80K,...,MS/MS count nt_rep2_03K,MS/MS count nt_rep2_06K,MS/MS count nt_rep2_12K,MS/MS count nt_rep2_24K,MS/MS count nt_rep2_80K,MS/MS count nt_rep3_03K,MS/MS count nt_rep3_06K,MS/MS count nt_rep3_12K,MS/MS count nt_rep3_24K,MS/MS count nt_rep3_80K
Protein IDs,Majority protein IDs,Protein names,Gene names,Q-value,Score,Only identified by site,Reverse,Potential contaminant,id,,,,,,,,,,,,,,,,,,,,,
A0A096LP01,A0A096LP01,NaN,LINC00493,0.000156,3.1709,NaN,NaN,NaN,0,175000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
P0DPI2;A0A0B4J2D5;P0DPI2-2;A0A0B4J2D5-2,P0DPI2;A0A0B4J2D5;P0DPI2-2;A0A0B4J2D5-2,NaN,NaN,0.000000,323.3100,NaN,NaN,NaN,1,16030000000,5810700000,502990000,0,0,15898000000,9895400000,149290000,0,0,...,12,9,2,0,0,14,11,5,0,0
A0A0B4J2F0,A0A0B4J2F0,NaN,PIGBOS1,0.000155,3.0551,NaN,NaN,NaN,2,0,0,0,0,0,0,0,0,0,0,...,1,2,1,0,0,0,1,0,0,0
A0A0B4J2F2;P57059,A0A0B4J2F2;P57059,Serine/threonine-protein kinase SIK1,LOC102724428;SIK1,0.000743,2.1973,NaN,NaN,NaN,3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0A0U1RRE5,A0A0U1RRE5,NaN,NaN,0.000000,29.0970,NaN,NaN,NaN,4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
REV__Q9UMQ3,REV__Q9UMQ3,NaN,NaN,1.000000,-2.0000,+,+,NaN,7607,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
REV__Q9Y297-2;REV__Q9Y297,REV__Q9Y297-2;REV__Q9Y297,NaN,NaN,1.000000,-2.0000,+,+,NaN,7608,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
REV__Q9Y4K0,REV__Q9Y4K0,NaN,NaN,1.000000,-2.0000,+,+,NaN,7609,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [96]:
multiindex = pd.MultiIndex.from_arrays(
        arrays=[
            [[s for s in sets_LFQ if col.startswith(s)][0] for col in df_index.columns],            
            #[[s for s in sets_SILAC if col.startswith(s)][0] for col in df_index.columns],
            
            #[re.match(name_pattern_SILAC, col).group("sets") for col in df_index.columns],  
            #[re.match(name_pattern_SILAC, col) for s in sets_SILAC]
            
            #[re.match(name_pattern_SILAC, col).group("rep") for col in df_index.columns],
            ["_".join(re.match(name_pattern_LFQ, col).group("cond", "rep")) for col in df_index.columns],
            
            #[fraction_dict[re.match(name_pattern_SILAC, col).group("frac")] for col in df_index.columns],
            [fraction_dict[re.match(name_pattern_LFQ, col).group("frac")] for col in df_index.columns],
            
            #[items.zfill(3) for items in list(list(zip(*[fraction_dict[re.match(name_pattern, col).group("frac")] for col in df_index.columns]))[0])],
        ],
        names=["Set", "Map", "Fraction"]
            )
df_index.columns = multiindex
df_index.sort_index(1, inplace=True)
df_index

Set                                                                                                                                                                                                                                        LFQ intensity  \
Map                                                                                                                                                                                                                                             EGF_rep1   
Fraction                                                                                                                                                                                                                                             03K   
Protein IDs                                        Majority protein IDs                               Protein names                        Gene names        Q-value  Score     Only identified by site Reverse Potential contaminant id                   
A0A096LP01                                         A0A096LP01                                         NaN                                  LINC00493         0.000156  3.1709   NaN                     NaN     NaN                   0        175000000   
P0DPI2;A0A0B4J2D5;P0DPI2-2;A0A0B4J2D5-2            P0DPI2;A0A0B4J2D5;P0DPI2-2;A0A0B4J2D5-2            NaN                                  NaN               0.000000  323.3100 NaN                     NaN     NaN                   1      16030000000   
A0A0B4J2F0                                         A0A0B4J2F0                                         NaN                                  PIGBOS1           0.000155  3.0551   NaN                     NaN     NaN                   2                0   
A0A0B4J2F2;P57059                                  A0A0B4J2F2;P57059                                  Serine/threonine-protein kinase SIK1 LOC102724428;SIK1 0.000743  2.1973   NaN                     NaN     NaN                   3                0   
A0A0U1RRE5                                         A0A0U1RRE5                                         NaN                                  NaN               0.000000  29.0970  NaN                     NaN     NaN                   4                0   
...                                                                                                                                                                                                                                                  ...   
REV__Q9UMQ3                                        REV__Q9UMQ3                                        NaN                                  NaN               1.000000 -2.0000   +                       +       NaN                   7607             0   
REV__Q9Y297-2;REV__Q9Y297                          REV__Q9Y297-2;REV__Q9Y297                          NaN                                  NaN               1.000000 -2.0000   +                       +       NaN                   7608             0   
REV__Q9Y4K0                                        REV__Q9Y4K0                                        NaN                                  NaN               1.000000 -2.0000   +                       +       NaN                   7609             0   
REV__Q9Y603-8;REV__Q9Y603-3;REV__Q9Y603-6;REV__... REV__Q9Y603-8;REV__Q9Y603-3;REV__Q9Y603-6;REV__... NaN                                  NaN               0.007095  1.3962   NaN                     +       NaN                   7610             0   
REV__Q9Y606-2;REV__Q9Y606                          REV__Q9Y606-2;REV__Q9Y606                          NaN                                  NaN               0.008318  1.3144   NaN                     +       NaN                   7611             0   

Set                                                                                                                                                                                                                        

In [97]:
dfi_1 = df_index.xs(
        np.nan, 0, "Reverse").xs(
        np.nan, 0, "Potential contaminant").xs(
        np.nan, 0, "Only identified by site")
dfi_1.shape



#list_drop_col = [column for column in df_index.columns if not column.startswith("Ratio ")]
#list_endcount = [column for column in df_index.columns if column.endswith("count")]
#list_endpct = [column for column in df_index.columns if column.endswith("[%]")]
#
#list_drop_col.extend(list_endcount)
#list_drop_col.extend(list_endpct)
#
#df_index = df_index.drop(list_drop_col, axis=1)

(7054, 60)

In [53]:
def stringency_silac(df_index):
    """The multiindex dataframe is subjected to stringency filtering. Only Proteins with complete profiles are
    considered (a set of f.e. 5 SILAC ratios in case you have 5 fractions / any proteins with missing values
    were rejected). Proteins were retained with 3 or more quantifications in each subfraction (=count). Furthermore,
    proteins with only 2 quantification events in one or more subfraction were retained, if their ratio
    variability for ratios obtained with 2 quantification events was below 30% (=var).
    SILAC ratios were linearly normalized by division through the fraction median. Subsequently normalization
    to SILAC loading was performed.

    Args:
        df_index: multiindex dataframe, which contains 3 level labels: MAP, Fraction, Type

    Returns:
        df_stringency_mapfracstacked: dataframe, in which "MAP" and "Fraction" are stacked;
        the columns "Ratio H/L count", "Ratio H/L variability [%]", and "Ratio H/L" stored as single level indices
    """

    # Fraction and Map will be stacked
    df_stack = df_index.stack(["Fraction", 'Map'])

    len_fractions = len(df_stack.index.get_level_values("Fraction").unique())

    # filtering for sufficient number of quantifications (count in 'Ratio H/L count'), taken
    # variability (var in Ratio H/L variability [%]) into account
    # zip: allows direct comparison of count and var
    # only if the filtering parameters are fulfilled the data will be introduced into df_countvarfiltered_stacked
    df_countvarfiltered_stacked = df_stack.loc[[count >= 3 or (count >= 2 and var < 30) for var, count in
                                                zip(df_stack["Ratio H/L variability [%]"],
                                                    df_stack['Ratio H/L count'])]]

    # "Ratio H/L":normalization to SILAC loading, each individual experiment (FractionXMap) will be divided by its median
    # np.median([...]): only entries, that are not NANs are considered
    df_normsilac_stacked = df_countvarfiltered_stacked["Ratio H/L"].unstack(["Fraction", "Map"]).apply(
        lambda x: x / np.median([el for el in x if not np.isnan(el)]), axis=0).stack(["Map", "Fraction"])

    df_stringency_mapfracstacked = df_countvarfiltered_stacked[["Ratio H/L count",
                                                            "Ratio H/L variability [%]"]].join(
        pd.DataFrame(df_normsilac_stacked, columns=["Ratio H/L"]))

    # dataframe is grouped (Map, id), that allows the filtering for complete profiles
    df_stringency_mapfracstacked = df_stringency_mapfracstacked.groupby(["Map", "id"]).filter(
        lambda x: len(x) >= len_fractions)

    # Ratio H/L is converted into Ratio L/H
    df_stringency_mapfracstacked["Ratio H/L"] = df_stringency_mapfracstacked["Ratio H/L"].transform(lambda x: 1 / x)


    return df_stringency_mapfracstacked

In [66]:
dfi_1

Set                                                                                                                                                      LFQ intensity  \
Map                                                                                                                                                               Map3   
Fraction                                                                                                                                                           01K   
Protein IDs                                        Majority protein IDs     Protein names                             Gene names Q-value  Score    id                    
A0A0B4J2F0                                         A0A0B4J2F0               NaN                                       PIGBOS1    0.000000 5.1353   1          32722000   
A0AV96;A0AV96-2;Q8TBY0;Q8TBY0-2;Q8TBY0-3;Q9NQ94... A0AV96;A0AV96-2          RNA-binding protein 47                    RBM47      0.000000 239.9100 2        1328000000   
A0AVF1;A0AVF1-3;A0AVF1-2                           A0AVF1;A0AVF1-3;A0AVF1-2 Intraflagellar transport protein 56       TTC26      0.000000 121.8700 3                 0   
A0AVI4;A0AVI4-2                                    A0AVI4;A0AVI4-2          E3 ubiquitin-protein ligase TM129         TMEM129    0.000000 21.5100  4                 0   
A0AVK6                                             A0AVK6                   Transcription factor E2F8                 E2F8       0.001619 2.1780   5           8414300   
...                                                                                                                                                                ...   
Q9Y6X3-2                                           Q9Y6X3-2                 MAU2 chromatid cohesion factor homolog    MAU2       0.006038 1.5769   10034      31152000   
Q9Y6X8                                             Q9Y6X8                   Zinc fingers and homeoboxes protein 2     ZHX2       0.000000 108.7600 10035    1644300000   
Q9Y6X9;Q9Y6X9-2                                    Q9Y6X9;Q9Y6X9-2          MORC family CW-type zinc finger protein 2 MORC2      0.000000 166.5600 10036    3666600000   
Q9Y6Y0                                             Q9Y6Y0                   Influenza virus NS1A-binding protein      IVNS1ABP   0.000000 211.4900 10037             0   
Q9Y6Y8;Q9Y6Y8-2                                    Q9Y6Y8;Q9Y6Y8-2          SEC23-interacting protein                 SEC23IP    0.000000 268.9500 10038     531210000   

Set                                                                                                                                                                  \
Map                                                                                                                                                                   
Fraction                                                                                                                                                        03K   
Protein IDs                                        Majority protein IDs     Protein names                             Gene names Q-value  Score    id                 
A0A0B4J2F0                                         A0A0B4J2F0               NaN                                       PIGBOS1    0.000000 5.1353   1      283720000   
A0AV96;A0AV96-2;Q8TBY0;Q8TBY0-2;Q8TBY0-3;Q9NQ94... A0AV96;A0AV96-2          RNA-binding protein 47                    RBM47      0.000000 239.9100 2      587030000   
A0AVF1;A0AVF1-3;A0AVF1-2                           A0AVF1;A0AVF1-3;A0AVF1-2 Intraflagellar transport protein 56       TTC26      0.000000 121.8700 3              0   
A0AVI4;A0AVI4-2                                    A0AVI4;A0AVI4-2          E3 ubiquitin-protein ligase TM129         TMEM129    0.000000 21.5100  4              0   
A0AVK6                                             A0AVK6                   Transcription factor E2F8            

In [56]:
df_stringency_mapfracstacked = stringency_silac(dfi_1)

In [58]:
df_stringency_mapfracstacked

Ratio H/L count  \
Q-value Score  Protein IDs                                        Majority protein IDs Protein names             Gene names id   Fraction Map                     
0.0     282.91 A0AV96;A0AV96-2;Q9NQ94;Q9NQ94-5;Q9NQ94-6;Q9NQ94... A0AV96;A0AV96-2      RNA-binding protein 47    RBM47      1    03K      MAP1                8   
                                                                                                                                          MAP2               11   
                                                                                                                                          MAP3               10   
                                                                                                                                          MAP4               14   
                                                                                                                                          MAP5               10   
...                                                                                                                                                         ...   
        323.31 Q9Y6Y8;Q9Y6Y8-2                                    Q9Y6Y8;Q9Y6Y8-2      SEC23-interacting protein SEC23IP    9887 80K      MAP2               33   
                                                                                                                                          MAP3               44   
                                                                                                                                          MAP4               36   
                                                                                                                                          MAP5               29   
                                                                                                                                          MAP6               33   

                                                                                                                                                Ratio H/L variability [%]  \
Q-value Score  Protein IDs                                        Majority protein IDs Protein names             Gene names id   Fraction Map                               
0.0     282.91 A0AV96;A0AV96-2;Q9NQ94;Q9NQ94-5;Q9NQ94-6;Q9NQ94... A0AV96;A0AV96-2      RNA-binding protein 47    RBM47      1    03K      MAP1                     12.069   
                                                                                                                                          MAP2                     17.206   
                                                                                                                                          MAP3                     25.194   
                                                                                                                                          MAP4                     34.456   
                                                                                                                                          MAP5                     76.902   
...                                                                                                                                                                   ...   
        323.31 Q9Y6Y8;Q9Y6Y8-2                                    Q9Y6Y8;Q9Y6Y8-2      SEC23-interacting protein SEC23IP    9887 80K      MAP2                     21.206   
                                                                                                                                          MAP3                     28.132   
                                                                                                                                          MAP4                     31.075   
                                                                                                                         

In [98]:
def stringency_lfq(df_index, selff):
    """The multiindex dataframe is subjected to stringency filtering. Only Proteins which were identified with
    at least [4] consecutive data points regarding the "LFQ intensity", and if summed MS/MS counts >= n(fractions)*[2]
    (LFQ5: min 10 and LFQ6: min 12, respectively; coverage filtering) were included.

    Args:
        df_index: multiindex dataframe, which contains 3 level labels: MAP, Fraction, Typ

    Returns:
        df_stringency_mapfracstacked: dataframe, in which "Map" and "Fraction" is stacked;
        "LFQ intensity" and "MS/MS count" define a single-level column index
        """

    # retrieve number of fractions that are present in the dataset
    df_fractionnumber_stacked = df_index.copy().stack("Fraction")
    number_fractions = len(df_fractionnumber_stacked.index.get_level_values("Fraction").unique())

    df_index = df_index.stack("Map")

    # level 0 = Fraction, level 1 = Type is converted into level 0 = Type, level 1 = Fraction
    #df_index.columns = df_index.columns.swaplevel(0, 1)

    # sorting the level 0, in order to have LFQ intensity -	MS/MS count instead of continuous alternation
    df_index.sort_index(axis=1, level=0, inplace=True)

    # "MS/MS count"-column: take the sum over the fractions;
    # if the sum is larger than n[fraction]*2, it will be stored in the new dataframe
    df_mscount_mapstacked = df_index.loc[df_index[('MS/MS count')].apply(np.sum, axis=1) >= (
            number_fractions * selff.summed_MSMS_counts)]

    df_stringency_mapfracstacked = df_mscount_mapstacked.copy()

    # series no dataframe is generated; if there are at least i.e. 4 consecutive non-NANs, data will be retained
    df_stringency_mapfracstacked = df_stringency_mapfracstacked.loc[
        df_stringency_mapfracstacked[("LFQ intensity")].apply(lambda x: any(
            np.invert(np.isnan(x)).rolling(window=selff.consecutive_LFQ_I).sum() >=
            selff.consecutive_LFQ_I), axis=1)]

    df_stringency_mapfracstacked = df_stringency_mapfracstacked.copy().stack("Fraction")

    return df_stringency_mapfracstacked

In [99]:
df_stringency_mapfracstacked = stringency_lfq(dfi_1, i_class)
df_stringency_mapfracstacked

Set                                                                                                                                                         LFQ intensity  \
Protein IDs                             Majority protein IDs                    Protein names             Gene names Q-value Score  id   Map      Fraction                  
P0DPI2;A0A0B4J2D5;P0DPI2-2;A0A0B4J2D5-2 P0DPI2;A0A0B4J2D5;P0DPI2-2;A0A0B4J2D5-2 NaN                       NaN        0.0     323.31 1    EGF_rep1 03K         16030000000   
                                                                                                                                                  06K          5810700000   
                                                                                                                                                  12K           502990000   
                                                                                                                                                  24K                   0   
                                                                                                                                                  80K                   0   
...                                                                                                                                                                   ...   
Q9Y6Y8;Q9Y6Y8-2                         Q9Y6Y8;Q9Y6Y8-2                         SEC23-interacting protein SEC23IP    0.0     323.31 7434 nt_rep3  03K           304640000   
                                                                                                                                                  06K           529930000   
                                                                                                                                                  12K          1625400000   
                                                                                                                                                  24K          3117900000   
                                                                                                                                                  80K          4855000000   

Set                                                                                                                                                         MS/MS count  
Protein IDs                             Majority protein IDs                    Protein names             Gene names Q-value Score  id   Map      Fraction               
P0DPI2;A0A0B4J2D5;P0DPI2-2;A0A0B4J2D5-2 P0DPI2;A0A0B4J2D5;P0DPI2-2;A0A0B4J2D5-2 NaN                       NaN        0.0     323.31 1    EGF_rep1 03K                17  
                                                                                                                                                  06K                11  
                                                                                                                                                  12K                 5  
                                                                                                                                                  24K                 0  
                                                                                                                                                  80K                 0  
...                                                                                                                                                                 ...  
Q9Y6Y8;Q9Y6Y8-2                         Q9Y6Y8;Q9Y6Y8-2                         SEC23-interacting protein SEC23IP    0.0     323.31 7434 nt_rep3  03K                 2  
                                                                                                                                                  06K                 4  
                                                  

Set                                                                                                                                              LFQ intensity  \
Protein IDs                                        Majority protein IDs Protein names             Gene names Q-value Score  id    Map  Fraction                  
A0AV96;A0AV96-2;Q8TBY0;Q8TBY0-2;Q8TBY0-3;Q9NQ94... A0AV96;A0AV96-2      RNA-binding protein 47    RBM47      0.0     239.91 2     Map3 01K          1328000000   
                                                                                                                                       03K           587030000   
                                                                                                                                       06K           899290000   
                                                                                                                                       12K          2816200000   
                                                                                                                                       24K          4567500000   
...                                                                                                                                                        ...   
Q9Y6Y8;Q9Y6Y8-2                                    Q9Y6Y8;Q9Y6Y8-2      SEC23-interacting protein SEC23IP    0.0     268.95 10038 Map6 06K           810280000   
                                                                                                                                       12K          1703700000   
                                                                                                                                       24K          3681400000   
                                                                                                                                       80K          6026600000   
                                                                                                                                       Cyt          4721900000   

Set                                                                                                                                              MS/MS count  
Protein IDs                                        Majority protein IDs Protein names             Gene names Q-value Score  id    Map  Fraction               
A0AV96;A0AV96-2;Q8TBY0;Q8TBY0-2;Q8TBY0-3;Q9NQ94... A0AV96;A0AV96-2      RNA-binding protein 47    RBM47      0.0     239.91 2     Map3 01K                24  
                                                                                                                                       03K                13  
                                                                                                                                       06K                15  
                                                                                                                                       12K                38  
                                                                                                                                       24K                47  
...                                                                                                                                                      ...  
Q9Y6Y8;Q9Y6Y8-2                                    Q9Y6Y8;Q9Y6Y8-2      SEC23-interacting protein SEC23IP    0.0     268.95 10038 Map6 06K                12  
                                                                                                                                       12K                32  
                                                                                                                                       24K                49  
                                                                                                                                       80K      